In [31]:
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Input
import numpy as np
import pickle
import os
from model import create_decoder,create_encoder
from constants import *
from model import *

In [32]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
# from constants import *


from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop, Adam
import keras

In [33]:
import numpy as np
from constants import *
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re
from nltk.corpus import stopwords

In [34]:
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Input
import numpy as np
import pickle
import os
from model import create_decoder,create_encoder
from constants import *
from model import *

In [35]:
MAX_LEN_TEXT = 150
MAX_LEN_SUMMARY = 100

In [36]:
trained_model = load_model(r'U:\nlp_project\text_summarization\artifacts\model\ts_model.pickle')

In [37]:
trained_model.save_weights(r"U:\nlp_project\text_summarization\artifacts\weights")

In [38]:
latent_dim = 500 
class create_encoder(keras.Model):
    def __init__(self,x_voc_size):
        super().__init__()
        self.encoder_emb = Embedding(x_voc_size, latent_dim,trainable=True,name="encoder_emb")
        self.encoder_lstm1 = LSTM(latent_dim,return_sequences=True,return_state=True,name="lstm1")
        self.dropout_one = Dropout((0.4),name="dropout_one")
        self.encoder_lstm2 = LSTM(latent_dim,return_sequences=True,return_state=True,name="lstm2")
        self.dropout_two = Dropout((0.2),name="dropout_two")
        self.encoder_lstm3=LSTM(latent_dim, return_state=True, return_sequences=True,name="lstm3")

    def call(self,inputs,*args, **kwargs):
        x = self.encoder_emb(inputs)
        encoder_output1, state_h1, state_c1 = self.encoder_lstm1(x)
        x = self.dropout_one(encoder_output1)
        encoder_output2, state_h2, state_c2 = self.encoder_lstm2(x)
        x = self.dropout_two(encoder_output2)
        encoder_outputs, state_h, state_c = self.encoder_lstm3(x)
        return encoder_outputs,state_h,state_c
    
class create_decoder(keras.Model):
    def __init__(self,y_voc_size):
        super().__init__()
        self.decoder_emb = Embedding(y_voc_size, latent_dim,trainable=True,name="decoder_emb")
        self.decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True,name="decoder_lstm") 
        self.dropout_decoder = Dropout((0.3),name="dropout_decoder")
        self.decoder_dense = TimeDistributed(Dense(y_voc_size, activation='softmax'))
        
    def call(self, decoder_inputs, context_vector=None):
        decoder_emb = self.decoder_emb(decoder_inputs)
        decoder_outputs,decoder_h_state, decoder_c_state = self.decoder_lstm(decoder_emb,initial_state=[context_vector[0], context_vector[1]])
        x = self.dropout_decoder(decoder_outputs)
        decoder_output = self.decoder_dense(x)
        return decoder_output,decoder_h_state, decoder_c_state

In [39]:
class Create_Encoder_Decoder():
    def __init__(self):
        self.encoder_inputs = Input(MAX_LEN_TEXT)
        self.decoder_inputs = Input(shape=(None,))
        self.x_voc_size = 27979
        self.y_voc_size = 7765
        self.trained_model = load_model(r'U:\nlp_project\text_summarization\artifacts\model\ts_model.pickle')


    
    def copy_weights(self,encoder,decoder):
        print("copy_weights start")
        loaded_layer_names = [layer.name for _,layer in enumerate(self.trained_model.layers)]

        #ENCODER
        for layer in encoder.layers:
            if layer.name in loaded_layer_names:
                index = loaded_layer_names.index(layer.name)
                print(index,layer.name)
                layer_weights = self.trained_model.layers[index].weights
                layer.set_weights(layer_weights)
                # if layer_weights:
                #     layer.set_weights(layer_weights)
        
        #DECODER
        for layer in decoder.layers:
            if layer.name in loaded_layer_names:
                index = loaded_layer_names.index(layer.name)
                layer_weights = self.trained_model.layers[index].get_weights()
                if layer_weights:
                    layer.set_weights(layer_weights)
        
        return encoder,decoder
                    


    def get_encoder_decoder(self):
        # CREATE ENCODER
        encoder = create_encoder(self.x_voc_size)
        encoder_outputs,state_h,state_c = encoder.call(self.encoder_inputs)
        encoder = Model(inputs=self.encoder_inputs,outputs=[encoder_outputs, state_h, state_c])
        context_vector = [state_h,state_c]


        # CREATE DECODER
        decoder = create_decoder(self.y_voc_size)
        decoder_output,decoder_h_state, decoder_c_state = decoder.call(self.decoder_inputs,context_vector)
        decoder = Model(inputs = [self.decoder_inputs] + context_vector, outputs=[decoder_output, decoder_h_state, decoder_c_state])

        # COPY WEIGHTS:
        encoder_model,decoder_model = self.copy_weights(encoder,decoder)

        return encoder_model,decoder_model

In [40]:
class get_tokenizer():
    def __init__(self):
        self.x_tokeizer_path = os.path.join(r"U:\nlp_project\text_summarization\artifacts\tokenizer\x" , 'x_tokenizer.pkl')
        self.y_tokenizer_path = os.path.join(r"U:\nlp_project\text_summarization\artifacts\tokenizer\y" , 'y_tokenizer.pkl')


    def get_tokenizers(self):
        with open(self.x_tokeizer_path, 'rb') as file:
            x_tokenizer = pickle.load(file)

        with open(self.y_tokenizer_path, 'rb') as file:
            y_tokenizer = pickle.load(file)
        
        return x_tokenizer,y_tokenizer

In [41]:
class make_prediction():
    def __init__(self,x_tokenizer,y_tokenizer,encoder,decoder):
        self.x_tokenizer = x_tokenizer
        self.y_tokenizer = y_tokenizer
        self.reverse_source_word_index = x_tokenizer.index_word
        self.reverse_target_word_index = y_tokenizer.index_word 
        self.target_word_index = y_tokenizer.word_index
        self.stop_words = set(stopwords.words('english'))
        self.encoder = encoder
        self.decoder = decoder


    def decode_sequence(self,input_seq):
        e_out, e_h, e_c = self.encoder.predict(input_seq)
        print(e_out)
        print("\n")
        print(e_h)
        print(e_c)
        context_vector = [e_h, e_c]
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = self.target_word_index['start']

        stop_condition = False
        decoded_sentence = ''
        while not stop_condition:
            output_tokens, h, c = self.decoder.predict([target_seq] + [e_h, e_c]) # inputs = [decoder_inputs] + context_vector
            sampled_token_index = np.argmax(output_tokens[0, -1, :])

            if sampled_token_index == 0:
                stop_condition = True
            else:
                sampled_token = self.reverse_target_word_index[sampled_token_index]

            if(sampled_token!='end'):
                decoded_sentence += ' '+sampled_token
                # print(decoded_sentence)
                if (sampled_token == 'end' or len(decoded_sentence.split()) >= (MAX_LEN_SUMMARY-1)):
                    stop_condition = True
            target_seq = np.zeros((1,1))
            target_seq[0, 0] = sampled_token_index
            e_h, e_c = h, c

        return decoded_sentence

    
    def predict(self,text):
        text = self.x_tokenizer.texts_to_sequences([text])
        text = pad_sequences(text, maxlen=MAX_LEN_TEXT, padding='post')
        # print(text)
        # print(len(text[0]))
        result = self.decode_sequence(text[0].reshape(1,MAX_LEN_TEXT))
        print(result)
        return result

In [42]:
ec_obj = Create_Encoder_Decoder()
encoder,decoder = ec_obj.get_encoder_decoder()

copy_weights start
1 encoder_emb
2 lstm1
3 dropout_one
4 lstm2
6 dropout_two
8 lstm3


In [43]:
encoder,decoder

(<keras.engine.functional.Functional at 0x2342ee04f10>,
 <keras.engine.functional.Functional at 0x2331c689a90>)

In [44]:
tobj = get_tokenizer()
x_tokenizer,y_tokenizer = tobj.get_tokenizers()

In [45]:
x_val = pd.read_csv(r"U:\nlp_project\text_summarization\artifacts\data\preprocessed_data\x_val.csv")

In [46]:
for text in x_val["cleaned_text"][:10]:
    print("Text:",text)
    predict = make_prediction(x_tokenizer,y_tokenizer,encoder,decoder)
    r = predict.predict(text)
    break

Text: drink lot coffee refuse drink without creamer subscribe save service blessing always creamer ready fridge much cheaper price delivery several reviewers said busted leaked make mess first got mine leaked box none busted leaked removed mine box second time careful removed slowly hand dump container keep mine btw put per cup
1/1 [==============================] - 1s 745ms/step
[[[-1.45375235e-02  3.53976749e-02 -1.40308840e-02 ...  2.56769895e-03
   -2.06912495e-03 -2.38827132e-02]
  [-3.97278927e-02  1.78722978e-01 -6.98727816e-02 ... -4.14660014e-02
    2.33242195e-02 -1.02452539e-01]
  [-3.78654674e-02  4.46361631e-01 -1.35012746e-01 ... -2.21830055e-01
    2.59278975e-02 -1.41147017e-01]
  ...
  [ 7.63157845e-01 -4.50108462e-04  1.24926728e-05 ...  3.14859026e-05
   -3.40677856e-04  1.78018774e-04]
  [ 7.63157785e-01 -4.50111664e-04  1.24926464e-05 ...  3.14855897e-05
   -3.40676663e-04  1.78018527e-04]
  [ 7.63157845e-01 -4.50108753e-04  1.24926428e-05 ...  3.14858662e-05
   -3